<a href="https://colab.research.google.com/github/briancoding/WhoDoYouLookLike/blob/main/%EC%96%BC%EA%B5%B4%EA%B0%90%EC%A0%95%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam  
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# initialize image data generator with rescaling

In [3]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
# preprocess all test images

In [5]:
train_generator = train_data_gen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/EmotionData/train',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 9853 images belonging to 7 classes.


In [6]:
#preprocess all train images

In [7]:
validation_generator=validation_data_gen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/EmotionData/test',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 7178 images belonging to 7 classes.


In [8]:
#create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001,decay=1e-6),metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
#Train the neural network/model

In [10]:
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=9296 // 64,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=7178//64
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
145/145 [==============================] - 2628s 18s/step - loss: 1.7777 - accuracy: 0.3412 - val_loss: 1.9026 - val_accuracy: 0.1334
Epoch 2/10
145/145 [==============================] - 102s 703ms/step - loss: 1.7493 - accuracy: 0.3470 - val_loss: 1.9069 - val_accuracy: 0.1336
Epoch 3/10
145/145 [==============================] - 35s 239ms/step - loss: 1.6992 - accuracy: 0.3550 - val_loss: 1.8424 - val_accuracy: 0.2263
Epoch 4/10
145/145 [==============================] - 32s 222ms/step - loss: 1.6380 - accuracy: 0.3904 - val_loss: 1.7894 - val_accuracy: 0.2676
Epoch 5/10
145/145 [==============================] - 32s 222ms/step - loss: 1.5791 - accuracy: 0.4273 - val_loss: 1.7561 - val_accuracy: 0.2782
Epoch 6/10
145/145 [==============================] - 32s 223ms/step - loss: 1.5218 - accuracy: 0.4464 - val_loss: 1.6442 - val_accuracy: 0.3443
Epoch 7/10
145/145 [==============================] - 32s 222ms/step - loss: 1.4909 - accuracy: 0.4595 - val_loss: 1.6555 - val_a

In [11]:
#save model structure in jason file

In [12]:
model_json = emotion_model.to_json()
with open('emotion_model.json','w') as json_file:
  json_file.write(model_json)

emotion_model.save_weights('emotion_model.h5')

In [13]:
# Test Emotion Detector
! git clone https://github.com/datamagic2020/Emotion_detection_with_CNN.git

Cloning into 'Emotion_detection_with_CNN'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [14]:
import cv2
import numpy as np
from keras.models import model_from_json

In [15]:
emotion_dict={0:'Angry',1:'Disgusted',2:'Fearful',3:'Happy',4:'Neutral',5:'Sad',6:'Surprised'}

In [16]:
#load json and create model

In [18]:
json_file = open('/content/emotion_model.json','r')
loaded_model_json= json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [19]:
#load weights into new model

In [21]:
emotion_model.load_weights('/content/emotion_model.h5')
print('Loaded modedl from disk')

Loaded modedl from disk


In [31]:
#Start the webcam feed
cap = cv2.VideoCapture(0)

In [30]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [37]:
from google.colab.patches import cv2_imshow

In [38]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4534  100  4534    0     0   210k      0 --:--:-- --:--:-- --:--:--  210k


In [ ]:
cap = cv2.VideoCapture('여기에 비디오 링크 넣기')
while True:
  ret, frame = cap.read()
  frame = cv2.resize(frame, (720,720))
  if not ret:
    break
  face_detector = cv2.CascadeClassifier('/content/Emotion_detection_with_CNN/haarcascades/haarcascade_frontalface_default.xml')
  gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

  num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

  for (x,y,w,h) in num_faces:
    cv2.rectangle(frame, (x,y-50),(x+w,y+h+10),(0,255,0),4)
    roi_gray_frame =gray_frame[y:y +h,x:x+w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame,(48,48)),-1),0)

    #prediction
    emotion_prediction = emotion_model.predict(cropped_img)
    maxindex = int(np.argmax(emotion_prediction))
    cv2.putText(frame,emotion_dict[maxindex],(x+5,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)

  cv2_imshow(frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()